# Análise das tendências políticas no município do Rio de Janeiro
## Por Omar Mesquita

Esse estudo tem por objetivo analisar as tendências políticas para escolha do cargo de Presidência da República no município do Rio de Janeiro. Além disso, será feita uma análise de como o perfil do eleitorado médio carioca pode ou não influenciar a escolha do cargo.

### Obtenção dos dados
Para estudar a cidade do Rio de Janeiro, obteve-se o código do município (**3304557**) pelo
[site do IBGE.](https://www.ibge.gov.br/cidades-e-estados/rj/rio-de-janeiro.html)<sup>1</sup>

[Pelo banco de dados do TSE,](https://www.tse.jus.br/servicos-eleitorais/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais)<sup>2</sup> obtivemos as zonas eleitorais no **estado** do Rio de Janeiro.


Por fim, utilizando a API do [Base dos Dados](https://basedosdados.org/)<sup>3</sup>, fez-se uma consulta SQL ao banco de dados para que fosse obtida a lista de zonas eleitorais no município do Rio de Janeiro: <br><br>

```SQL

    SELECT ano, turno, sigla_uf, id_municipio, zona, numero_partido, resultado, votos
    FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
    WHERE cargo='presidente'
      AND ano = 2022
      AND sigla_uf='RJ'
      AND id_municipio = '3304557'
    LIMIT 100
```


In [ ]:
# Por favor, execute essa célula antes de continuar
# a leitura para garantir as importações de módulos

import pandas as pd     # Leitura e tratamento de tabelas
import basedosdados as bd   # API para acessar dados do TSE
import os                   # Caminhos e I/O

# Determina o diretório no qual o caderno está sendo executado
currentDir = os.getcwd()

Devido à elevada quantidade de zonas eleitorais no município do Rio de Janeiro, faz-se necessário escolher aquelas que contenham a maior quantidade de pessoas votantes. Para tanto, foi baixada uma tabela em formato `.csv` contendo dados da última eleição (**outubro de 2022**) no **estado do Rio de Janeiro (RJ)** pelo site de [estatísticas do eleitorado](https://www.tse.jus.br/eleicoes/estatisticas/estatisticas-de-eleitorado/consulta-por-municipio-zona).<sup>2</sup>

Em seguida, foi feito o tratamento dos dados utilizando a biblioteca `pandas` para Python.

**Obs.:** A tabela foi baixada e, em seguida, salva em formato **UTF-8** para evitar problemas de codificação.

In [ ]:
# Armazena o arquivo .csv em um DataFrame
df = pd.read_csv(os.path.join(currentDir,"csv","quantidade_de_eleitores.csv"))

# Cria um DataFrame apenas com as colunas que serão utilizadas (Zona e Quantidade)
# e satisfazem a condição de estarem no município do Rio de Janeiro.
# Em seguida, ordena o DataFrame em ordem decrescente de quantidade de eleitores
# e seleciona as 4 zonas com maior quantidade de eleitores.
df_rj = df[ df["Município"] == "RIO DE JANEIRO"] [['Zona', 'Quantidade']].sort_values(by='Quantidade', ascending=False).head(4)

print(df_rj)

A partir da visualização da tabela, nota-se que as zonas eleitorais **119**, **9**, **179** e **214** são as que concentram maior quantidade de pessoas votantes.

Dessa forma, é possível, agora, fazer uma análise dos resultados eleitorais dessas zonas.

Para tanto, baixou-se uma tabela com informações sobre os resultados das eleições presidenciais de 2022 nas zonas eleitorais previamente citadas. Foi utilizada a API Python do [Base dos Dados](https://basedosdados.org/)<sup>3</sup> e uma consulta SQL no interior da função para se obter tais dados.

In [ ]:
# AVISO! Caso você tenha baixado esse caderno ou o repositório,
# não é necessário executar essa célula, pois os dados já foram baixados

# Baixando tabelas correspondentes aos resultados das eleições, gênero e instrução do eleitorado
zonas = [119, 9, 179, 214]

for i in zonas:

    bd.download(
        savepath=os.path.join(currentDir,"csv","resultados",f"resultados_eleicoes_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT turno, zona, numero_partido, resultado, votos, sigla_partido
        FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND cargo = "presidente"
          AND zona = {i}
        LIMIT 1000
        '''
    )

    bd.download(
        savepath=os.path.join(currentDir,"csv","genero",f"genero_eleitorado_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT genero, eleitores
        FROM `basedosdados.br_tse_eleicoes.perfil_eleitorado_secao`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND zona = {i}
        LIMIT 1000
        '''
    )

    bd.download(
        savepath=os.path.join(currentDir,"csv","instrucao",f"instrucao_eleitorado_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT instrucao, eleitores
        FROM `basedosdados.br_tse_eleicoes.perfil_eleitorado_secao`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND zona = {i}
        LIMIT 1000
        '''
    )

## Uma breve discussão sobre alinhamento político

Para dar sequência ao estudo, será necessário classificar os partidos com maior expressividade nas zonas eleitorais, isto é, aqueles presentes nas tabelas obtidas pela API do **Base dos Dados**.

Trata-se, no entanto, de um assunto delicado e complexo uma vez que o alinhamento político é um conceito que varia conforme o contexto histórico, socioeconômico e considera outros fatores. Por exemplo, dois partidos distintos podem se posicionar de forma semelhante a respeito de uma dada pauta A, mas terem posições opostas em relação à pauta B.

Assim, para fins de simplificação, será utilizada a classificação política feita pelo [Congresso em Foco](https://congressoemfoco.uol.com.br/area/pais/direita-cresce-e-engole-o-centro-no-congresso-mais-fragmentado-da-historia/)<sup>4</sup>, o qual classifica os partidos em esquerda, centro e direita.

Classificando os partidos:

- `PDT (12)`: Esquerda
- `PT (13)`: Esquerda
- `PTB (14)`: Direita
- `MDB (15)`: Centro
- `PSTU (16)`: Esquerda
- `PCB (21)`: Esquerda
- `PL (22)`: Direita
- `DC (27)`: Direita
- `NOVO (30)`: Direita
- `UNIÃO (44)`: Direita
- `UP (80)`: Esquerda

In [ ]:
# Criação de DataFrames para cada zona eleitoral
df9 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_9.csv"))
df119 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_119.csv"))
df179 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_179.csv"))
df214 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_214.csv"))

# Criação de arrays para cada alinhamento político
esquerda = [12, 13, 16, 80, 21]
direita = [14, 22, 30, 44, 27]
centro = [15]


In [ ]:
# Análise da zona eleitoral 9

# Inicializa variáveis para armazenar os votos de cada alinhamento político
esq_9 = 0
dir_9 = 0
centro_9 = 0

# Percorre os arrays de cada alinhamento político e soma os votos de cada partido
for i in esquerda:
    esq_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

for i in direita:
    dir_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

for i in centro:
    centro_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

print(f"Proporção de votos à esquerda: {esq_9/(esq_9+dir_9+centro_9) * 100:.1f}%")
print(f"Proporção de votos à direita: {dir_9/(esq_9+dir_9+centro_9) * 100:.1f}%")
print(f"Proporção de votos ao centro: {centro_9/(esq_9+dir_9+centro_9) * 100:.1f}%")

genero_9 = pd.read_csv(os.path.join(currentDir,"csv","genero","genero_eleitorado_9.csv"))

feminino_9 = 0
for i in genero_9.loc[genero_9["genero"] == "feminino", ["eleitores"]].values:
    if i == 'feminino':
        continue
    feminino_9 += int(i[0])

masculino_9 = 0
for i in genero_9.loc[genero_9["genero"] == "masculino", ["eleitores"]].values:
    if i == 'masculino':
        continue
    masculino_9 += int(i[0])

print(f"Proporção de pessoas do gênero feminino na seção: {feminino_9/(feminino_9+masculino_9) * 100:.1f}%")

instrucao_9 = pd.read_csv(os.path.join(currentDir,"csv","instrucao","instrucao_eleitorado_9.csv"))

analfabeto_9 = 0
fund_completo_9 = 0
fund_incompleto_9 = 0
medio_completo_9 = 0
medio_incompleto_9 = 0
superior_completo_9 = 0
superior_incompleto_9 = 0

for i in instrucao_9.loc[instrucao_9["instrucao"] == "analfabeto", ["eleitores"]].values:
    if i == 'analfabeto':
        continue
    analfabeto_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino fundamental completo", ["eleitores"]].values:
    if i == 'fundamental completo':
        continue
    fund_completo_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino fundamental incompleto", ["eleitores"]].values:
    if i == 'fundamental incompleto':
        continue
    fund_incompleto_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino medio completo", ["eleitores"]].values:
    if i == 'médio completo':
        continue
    medio_completo_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino medio incompleto", ["eleitores"]].values:
    if i == 'médio incompleto':
        continue
    medio_incompleto_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino superior completo", ["eleitores"]].values:
    if i == 'superior completo':
        continue
    superior_completo_9 += int(i[0])

for i in instrucao_9.loc[instrucao_9["instrucao"] == "ensino superior incompleto", ["eleitores"]].values:
    if i == 'superior incompleto':
        continue
    superior_incompleto_9 += int(i[0])

baixa_escolaridade_9 = analfabeto_9 + fund_completo_9 + fund_incompleto_9 + medio_incompleto_9
alta_escolaridade_9 = medio_completo_9 + superior_completo_9 + superior_incompleto_9

print(f'Proporção de pessoas com baixa escolaridade: {baixa_escolaridade_9/(baixa_escolaridade_9+alta_escolaridade_9) * 100:.1f}%')


In [ ]:
# Análise da zona eleitoral 119

# Inicializa variáveis para armazenar os votos de cada alinhamento político
esq_119 = 0
dir_119 = 0
centro_119 = 0

for i in esquerda:
    esq_119 += df119.loc[df119["numero_partido"] == i, ["votos"]].values[0][0]

for i in direita:
    dir_119 += df119.loc[df119["numero_partido"] == i, ["votos"]].values[0][0]

for i in centro:
    centro_119 += df119.loc[df119["numero_partido"] == i, ["votos"]].values[0][0]


print(f"Proporção de votos à esquerda: {esq_119/(esq_119+dir_119+centro_119) * 100:.1f}%")
print(f"Proporção de votos à direita: {dir_119/(esq_119+dir_119+centro_119) * 100:.1f}%")
print(f"Proporção de votos ao centro: {centro_119/(esq_119+dir_119+centro_119) * 100:.1f}%")

genero_119 = pd.read_csv(os.path.join(currentDir,"csv","genero","genero_eleitorado_119.csv"))

feminino_119 = 0
for i in genero_119.loc[genero_119["genero"] == "feminino", ["eleitores"]].values:
    if i == 'feminino':
        continue
    feminino_119 += int(i[0])

masculino_119 = 0
for i in genero_119.loc[genero_119["genero"] == "masculino", ["eleitores"]].values:
    if i == 'masculino':
        continue
    masculino_119 += int(i[0])

#print("Pessoas do gênero feminino", feminino_119)
#print("Pessoas do gênero masculino", masculino_119)
print(f"Proporção de pessoas do gênero feminino na seção: {feminino_119/(feminino_119+masculino_119) * 100:.1f}%")

instrucao_119 = pd.read_csv(os.path.join(currentDir,"csv","instrucao","instrucao_eleitorado_119.csv"))

analfabeto_119 = 0
fund_completo_119 = 0
fund_incompleto_119 = 0
medio_completo_119 = 0
medio_incompleto_119 = 0
superior_completo_119 = 0
superior_incompleto_119 = 0

for i in instrucao_119.loc[instrucao_119["instrucao"] == "analfabeto", ["eleitores"]].values:
    if i == 'analfabeto':
        continue
    analfabeto_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino fundamental completo", ["eleitores"]].values:
    if i == 'fundamental completo':
        continue
    fund_completo_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino fundamental incompleto", ["eleitores"]].values:
    if i == 'fundamental incompleto':
        continue
    fund_incompleto_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino medio completo", ["eleitores"]].values:
    if i == 'médio completo':
        continue
    medio_completo_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino medio incompleto", ["eleitores"]].values:
    if i == 'médio incompleto':
        continue
    medio_incompleto_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino superior completo", ["eleitores"]].values:
    if i == 'superior completo':
        continue
    superior_completo_119 += int(i[0])

for i in instrucao_119.loc[instrucao_119["instrucao"] == "ensino superior incompleto", ["eleitores"]].values:
    if i == 'superior incompleto':
        continue
    superior_incompleto_119 += int(i[0])

baixa_escolaridade_119 = analfabeto_119 + fund_completo_119 +fund_incompleto_119 + medio_incompleto_119
alta_escolaridade_119 = medio_completo_119 + superior_completo_119 + superior_incompleto_119

print(f'Proporção de pessoas com baixa escolaridade: {baixa_escolaridade_119/(baixa_escolaridade_119+alta_escolaridade_119) * 100:.1f}%')

In [ ]:
# Análise da zona eleitoral 179

# Inicializa variáveis para armazenar os votos de cada alinhamento político
esq_179 = 0
dir_179 = 0
centro_179 = 0

for i in esquerda:
    esq_179 += df179.loc[df179["numero_partido"] == i, ["votos"]].values[0][0]

for i in direita:
    dir_179 += df179.loc[df179["numero_partido"] == i, ["votos"]].values[0][0]

for i in centro:
    centro_179 += df179.loc[df179["numero_partido"] == i, ["votos"]].values[0][0]

print(f"Proporção de votos à esquerda: {esq_179/(esq_179+dir_179+centro_179) * 100:.1f}%")
print(f"Proporção de votos à direita: {dir_179/(esq_179+dir_179+centro_179) * 100:.1f}%")
print(f"Proporção de votos ao centro: {centro_179/(esq_179+dir_179+centro_179) * 100:.1f}%")

genero_179 = pd.read_csv(os.path.join(currentDir,"csv","genero","genero_eleitorado_179.csv"))

feminino_179 = 0

for i in genero_179.loc[genero_179["genero"] == "feminino", ["eleitores"]].values:
    if i == 'feminino':
        continue
    feminino_179 += int(i[0])

masculino_179 = 0

for i in genero_179.loc[genero_179["genero"] == "masculino", ["eleitores"]].values:
    if i == 'masculino':
        continue
    masculino_179 += int(i[0])

print(f"Proporção de pessoas do gênero feminino na seção: {feminino_179/(feminino_179+masculino_179) * 100:.1f}%")

instrucao_179 = pd.read_csv(os.path.join(currentDir,"csv","instrucao","instrucao_eleitorado_179.csv"))

analfabeto_179 = 0
fund_completo_179 = 0
fund_incompleto_179 = 0
medio_completo_179 = 0
medio_incompleto_179 = 0
superior_completo_179 = 0
superior_incompleto_179 = 0

for i in instrucao_179.loc[instrucao_179["instrucao"] == "analfabeto", ["eleitores"]].values:
    if i == 'analfabeto':
        continue
    analfabeto_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino fundamental completo", ["eleitores"]].values:
    if i == 'fundamental completo':
        continue
    fund_completo_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino fundamental incompleto", ["eleitores"]].values:
    if i == 'fundamental incompleto':
        continue
    fund_incompleto_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino medio completo", ["eleitores"]].values:
    if i == 'médio completo':
        continue
    medio_completo_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino medio incompleto", ["eleitores"]].values:
    if i == 'médio incompleto':
        continue
    medio_incompleto_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino superior completo", ["eleitores"]].values:
    if i == 'superior completo':
        continue
    superior_completo_179 += int(i[0])

for i in instrucao_179.loc[instrucao_179["instrucao"] == "ensino superior incompleto", ["eleitores"]].values:
    if i == 'superior incompleto':
        continue
    superior_incompleto_179 += int(i[0])

baixa_escolaridade_179 = analfabeto_179 + fund_completo_179 +fund_incompleto_179 + medio_incompleto_179
alta_escolaridade_179 = medio_completo_179 + superior_completo_179 + superior_incompleto_179

print(f'Proporção de pessoas com baixa escolaridade: {baixa_escolaridade_179/(baixa_escolaridade_179+alta_escolaridade_179) * 100:.1f}%')

In [ ]:
# Análise da zona eleitoral 214

# Inicializa variáveis para armazenar os votos de cada alinhamento político
esq_214 = 0
dir_214 = 0
centro_214 = 0

for i in esquerda:
    esq_214 += df214.loc[df214["numero_partido"] == i, ["votos"]].values[0][0]

for i in direita:
    dir_214 += df214.loc[df214["numero_partido"] == i, ["votos"]].values[0][0]

for i in centro:
    centro_214 += df214.loc[df214["numero_partido"] == i, ["votos"]].values[0][0]

print(f"Proporção de votos à esquerda: {esq_214/(esq_214+dir_214+centro_214) * 100:.1f}%")
print(f"Proporção de votos à direita: {dir_214/(esq_214+dir_214+centro_214) * 100:.1f}%")
print(f"Proporção de votos ao centro: {centro_214/(esq_214+dir_214+centro_214) * 100:.1f}%")

genero_214 = pd.read_csv(os.path.join(currentDir,"csv","genero","genero_eleitorado_214.csv"))

feminino_214 = 0

for i in genero_214.loc[genero_214["genero"] == "feminino", ["eleitores"]].values:
    if i == 'feminino':
        continue
    feminino_214 += int(i[0])

masculino_214 = 0

for i in genero_214.loc[genero_214["genero"] == "masculino", ["eleitores"]].values:
    if i == 'masculino':
        continue
    masculino_214 += int(i[0])

print(f"Proporção de pessoas do gênero feminino na seção: {feminino_214/(feminino_214+masculino_214) * 100:.1f}%")

instrucao_214 = pd.read_csv(os.path.join(currentDir,"csv","instrucao","instrucao_eleitorado_214.csv"))

analfabeto_214 = 0
fund_completo_214 = 0
fund_incompleto_214 = 0
medio_completo_214 = 0
medio_incompleto_214 = 0
superior_completo_214 = 0
superior_incompleto_214 = 0

for i in instrucao_214.loc[instrucao_214["instrucao"] == "analfabeto", ["eleitores"]].values:
    if i == 'analfabeto':
        continue
    analfabeto_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino fundamental completo", ["eleitores"]].values:
    if i == 'fundamental completo':
        continue
    fund_completo_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino fundamental incompleto", ["eleitores"]].values:
    if i == 'fundamental incompleto':
        continue
    fund_incompleto_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino medio completo", ["eleitores"]].values:
    if i == 'médio completo':
        continue
    medio_completo_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino medio incompleto", ["eleitores"]].values:
    if i == 'médio incompleto':
        continue
    medio_incompleto_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino superior completo", ["eleitores"]].values:
    if i == 'superior completo':
        continue
    superior_completo_214 += int(i[0])

for i in instrucao_214.loc[instrucao_214["instrucao"] == "ensino superior incompleto", ["eleitores"]].values:
    if i == 'superior incompleto':
        continue
    superior_incompleto_214 += int(i[0])

baixa_escolaridade_214 = analfabeto_214 + fund_completo_214 +fund_incompleto_214 + medio_incompleto_214
alta_escolaridade_214 = medio_completo_214 + superior_completo_214 + superior_incompleto_214

print(f'Proporção de pessoas com baixa escolaridade: {baixa_escolaridade_214/(baixa_escolaridade_214+alta_escolaridade_214) * 100:.1f}%')

# Conclusão

Agora, salvamos as saídas das células anteriores para que possamos analisar os resultados.

**Zona 9:**
```
Proporção de votos à esquerda: 38.7%
Proporção de votos à direita: 56.2%
Proporção de votos ao centro: 5.1%
Proporção de pessoas do gênero feminino na seção: 62.0%
Proporção de pessoas com baixa escolaridade: 28.0%
```

**Zona 119:**
```
Proporção de votos à esquerda: 41.8%
Proporção de votos à direita: 52.0%
Proporção de votos ao centro: 6.3%
Proporção de pessoas do gênero feminino na seção: 56.1%
Proporção de pessoas com baixa escolaridade: 32.7%
```

**Zona 179:**
```
Proporção de votos à esquerda: 54.4%
Proporção de votos à direita: 42.3%
Proporção de votos ao centro: 3.3%
Proporção de pessoas do gênero feminino na seção: 56.1%
Proporção de pessoas com baixa escolaridade: 65.3%
```

**Zona 214:**
```
Proporção de votos à esquerda: 53.0%
Proporção de votos à direita: 42.3%
Proporção de votos ao centro: 4.8%
Proporção de pessoas do gênero feminino na seção: 53.9%
Proporção de pessoas com baixa escolaridade: 52.6%
```

Primeiramente, devido ao escopo do projeto, não é possível analisar todas as zonas eleitorais do estado do Rio de Janeiro. Por isso, foram escolhidas as 4 (quatro) zonas com maior quantidade de pessoas votantes. Isso limita muito a análise, mas é possível, ainda assim, observar algumas tendências.

Por exemplo, as zonas 119 e 179, apesar de possuírem a mesma quantidade de mulheres votantes, apresentam uma **diferença de mais de 10% na proporção de votos à esquerda**. Esse dado pode ser explicado pelo fato de que a zona 179 possui uma **proporção significativamente maior de pessoas com baixa escolaridade**, o que pode explicar a tendência de votos à esquerda.

Esse comportamento se mantém em todas as zonas analisadas. A zona 9, por exemplo, possui uma **proporção de votos à esquerda de 38.7%**, enquanto a zona 214 possui uma de 53%. A diferença está, provavelmente, relacionada à elevada quantidade de pessoas de baixa escolaridade na zona 214 (aumento de 24.6% em relação à zona 9).

A preferência por partidos de esquerda por pessoas de baixa escolaridade pode ser explicada pelo fato de que, **em geral**, partidos esquerdistas são mais propensos a investir em políticas públicas de inclusão social como educação e saúde universais.

Outro fator de análise é a quantidade de pessoas do genêro feminino nos locais de votação. Essa variável gerou resultados inesperados na análise. Por exemplo, a zona 9 apresenta a maior porcentagem de mulheres votantes e, também, a menor adesão à esquerda. Esse fato pode ser considerado inesperado uma vez que, **em geral**, a esquerda é mais propensa a apoiar políticas públicas voltadas para o gênero feminino como a igualdade salarial e a descriminalização/legalização do aborto.


Por fim, um fenômeno interessante que pode ser notado é que, mesmo sendo representado por apenas um partido, o **centro mantém uma presença razoável nas zonas analisadas (entre 3% e 6% dos votos)**. O motivo dessa presença nessas zonas vai além da análise feita aqui, mas, pode-se especular que pelo fato dele estar representado por um partido **antigo e bem consolidado** na política brasileira (MDB), ele mantém presença nas urnas. Além disso, a **falta de uma ideologia clara** pode ter contribuído para esse fato.



# Referências bibliográficas

[1] [IBGE](https://www.ibge.gov.br/cidades-e-estados/rj/rio-de-janeiro.html)
[2] [TSE](https://www.tse.jus.br)
[3] [Base dos Dados](https://basedosdados.org)
[4] [Congresso em Foco](https://congressoemfoco.uol.com.br/area/pais/direita-cresce-e-engole-o-centro-no-congresso-mais-fragmentado-da-historia/)